# **Gesture X** - Virtual Drawing with Hand Gestures

GestureX is a computer vision-based virtual drawing application that lets users draw on the screen using only their hand gestures. Powered by OpenCV and MediaPipe, this project tracks hand landmarks in real-time using a webcam and allows users to switch between colours or an eraser by moving their hands to different areas of the screen.

# **Installation**

1. Python

Python
*(Installed version should be Python 3.11.9 as it is compatible with media pipe)*

Download it from: [Python 3.11.9 Download](https://www.python.org/downloads/release/python-3119/)

To check installation:
```
python --version
```



2. OpenCV, NumPy & MediaPipe

Execute the following commands in order:

```
python -m pip install --upgrade pip

pip install opencv-python

pip install numpy

pip install mediapipe
```
To confirm if everything works fine, open vscode and create a .py file and type:
```
import cv2
import numpy as np
import mediapipe as mp

print("OpenCV version:", cv2.__version__)
print("NumPy version:", np.__version__)
print("MediaPipe is working!")
```
If it runs without errors, your installation is fine.


# **HandTrackingModule.py**



```
import cv2
import mediapipe as mp
import math
import time
import numpy as np

class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode=self.mode,
                                max_num_hands=self.maxHands,
                                min_detection_confidence=self.detectionCon,
                                min_tracking_confidence=self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList, yList, bbox = [], [], []
        self.lmList = []

        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, _ = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []

        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        for id in range(1, 5):
            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        return fingers

    def findDistance(self, p1, p2, img, draw=True, r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)

        length = math.hypot(x2 - x1, y2 - y1)
        return length, img, [x1, y1, x2, y2, cx, cy]

```



# **Painter.py**



```
import cv2
import numpy as np
import os
import HandTrackingModule as htm

brushThickness = 15
eraserThickness = 50

# Load toolbar images
folderPath = "C:/Users/Username/Desktop/GestureX/Header"
myList = os.listdir(folderPath)
overlayList = [cv2.imread(f'{folderPath}/{imPath}') for imPath in myList]

header = overlayList[0]  # Default header
drawColor = (128, 0, 128)  # Set default to Purple

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

detector = htm.handDetector(detectionCon=0.65, maxHands=1)
xp, yp = 0, 0
imgCanvas = np.zeros((720, 1280, 3), np.uint8)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)

    img = detector.findHands(img)
    lmList, _ = detector.findPosition(img, draw=False)

    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]  # Index Finger Tip
        x2, y2 = lmList[12][1:] # Middle Finger Tip

        fingers = detector.fingersUp()

        # Selection Mode (Index + Middle Finger Up)
        if fingers[1] and fingers[2] and not fingers[3] and not fingers[4]:  
            if y1 < 120:
                if 668 < x1 < 780:
                    header = overlayList[0]
                    drawColor = (128, 0, 128)  # Fix - Correct Purple
                elif 815 < x1 < 923:
                    header = overlayList[1]
                    drawColor = (255, 0, 0)  # Fix - Correct Blue
                elif 956 < x1 < 1065:
                    header = overlayList[2]
                    drawColor = (0, 255, 0)  # Fix - Correct Green
                elif 1115 < x1 < 1200:
                    header = overlayList[3]
                    drawColor = (0, 0, 0)  # Eraser

            xp, yp = 0, 0  # Reset previous points to fix unwanted lines
            cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), drawColor, cv2.FILLED)

        # Erasing Mode (Full Hand Open)
        elif all(fingers):  
            cv2.circle(img, (x1, y1), 50, (0, 0, 0), cv2.FILLED)
            cv2.line(imgCanvas, (xp, yp), (x1, y1), (0, 0, 0), eraserThickness)
            xp, yp = x1, y1

        # Drawing Mode (Only Index Finger Up)
        elif fingers[1] and not fingers[2]:  
            cv2.circle(img, (x1, y1), 15, drawColor, cv2.FILLED)
            if xp == 0 and yp == 0:
                xp, yp = x1, y1  # Start new stroke
            cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)
            xp, yp = x1, y1
        else:
            xp, yp = 0, 0  # Reset points when fingers are lifted

    # Merge layers
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv, cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img, imgInv)
    img = cv2.bitwise_or(img, imgCanvas)

    img[0:120, 0:1280] = header
    cv2.imshow("Image", img)
    cv2.imshow("Canvas", imgCanvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
            break

```


